In [ ]:
from manim import *
import numpy as np

class PyramidAreaAndVolume(ThreeDScene):
    def construct(self):
        # ================= SETUP =================
        self.set_camera_orientation(phi=65 * DEGREES, theta=-25 * DEGREES, zoom=0.7)
        SHIFT_VEC = RIGHT * 3.5 # पिरामिडलाई अलिकति दायाँ राख्ने
        
        L, H = 3.0, 4.0
        
        # Vertices (Coordinates)
        apex = np.array([0, 0, H]) + SHIFT_VEC
        v1 = np.array([-L/2, -L/2, 0]) + SHIFT_VEC 
        v2 = np.array([ L/2, -L/2, 0]) + SHIFT_VEC 
        v3 = np.array([ L/2,  L/2, 0]) + SHIFT_VEC 
        v4 = np.array([-L/2,  L/2, 0]) + SHIFT_VEC 
        center_base = np.array([0, 0, 0]) + SHIFT_VEC
        
        # Pyramid Parts
        base_poly = Polygon(v1, v2, v3, v4, color=RED_D, fill_opacity=0.8, stroke_color=WHITE, stroke_width=2)
        face_style = {"fill_opacity": 0.6, "stroke_color": WHITE, "stroke_width": 2, "color": BLUE_D}
        s1 = Polygon(apex, v1, v2, **face_style) # Front
        s2 = Polygon(apex, v2, v3, **face_style) # Right
        s3 = Polygon(apex, v3, v4, **face_style) # Back
        s4 = Polygon(apex, v4, v1, **face_style) # Left
        
        pyramid_grp = VGroup(base_poly, s1, s2, s3, s4)

        # ================= PART 1: SURFACE AREA HUD =================
        # Title
        title = Text("Surface Area (सतहको क्षेत्रफल)", font_size=32, color=YELLOW, font="sans-serif")
        title.to_corner(UL, buff=0.5)
        self.add_fixed_in_frame_mobjects(title)
        
        hud_scale = 0.75
        line_buff = 0.15
        section_buff = 0.4
        
        # 1. Base Area
        t_base = Text("1. Base Area (आधार):", font_size=24, color=RED, font="sans-serif").next_to(title, DOWN, aligned_edge=LEFT, buff=section_buff)
        f_base = MathTex(r"A_{base} = L^2", color=RED).scale(hud_scale).next_to(t_base, DOWN, aligned_edge=LEFT, buff=line_buff)
        
        # 2. Triangles
        t_tri = Text("2. 4 Triangles (त्रिभुजहरू):", font_size=24, color=BLUE, font="sans-serif").next_to(f_base, DOWN, aligned_edge=LEFT, buff=section_buff)
        
        # a) One Triangle
        t_tri_one = Text("a) One Triangle:", font_size=20, font="sans-serif").next_to(t_tri, DOWN, aligned_edge=LEFT, buff=line_buff)
        f_tri_one = MathTex(r"Area = \frac{1}{2} L l", color=BLUE_A).scale(hud_scale).next_to(t_tri_one, DOWN, aligned_edge=LEFT, buff=line_buff)
        
        # b) Total 4 Triangles
        t_tri_four = Text("b) Total 4 Triangles:", font_size=20, font="sans-serif").next_to(f_tri_one, DOWN, aligned_edge=LEFT, buff=line_buff+0.1)
        
        # Calculation Steps for Animation
        calc_step1 = MathTex(r"4 \times \frac{1}{2} L l", color=BLUE).scale(hud_scale).next_to(t_tri_four, DOWN, aligned_edge=LEFT, buff=line_buff)
        calc_step2 = MathTex(r"= 2 L l", color=GREEN).scale(hud_scale).next_to(calc_step1, RIGHT, buff=0.2)
        
        # 3. Total Area
        t_total = Text("Total Area (जम्मा):", font_size=24, color=GREEN, font="sans-serif").next_to(calc_step1, DOWN, aligned_edge=LEFT, buff=section_buff)
        f_total = MathTex(r"TSA = L^2 + 2 L l", color=GREEN).scale(0.9).next_to(t_total, DOWN, aligned_edge=LEFT, buff=line_buff)

        hud_sa = VGroup(t_base, f_base, t_tri, t_tri_one, f_tri_one, t_tri_four, t_total, f_total)
        self.add_fixed_in_frame_mobjects(hud_sa)
        hud_sa.set_opacity(0) # Hide initially

        # ================= ANIMATION: SURFACE AREA =================
        self.play(DrawBorderThenFill(pyramid_grp), run_time=1)
        
        # --- Base Area ---
        self.play(t_base.animate.set_opacity(1), f_base.animate.set_opacity(1), base_poly.animate.set_color(RED).set_opacity(1))
        self.wait(0.5)
        self.play(base_poly.animate.set_opacity(0.3))

        # --- Explosion ---
        explode_dist = 2.5
        self.play(t_tri.animate.set_opacity(1))
        self.play(
            s1.animate.shift(DOWN * explode_dist).set_z(0).set_opacity(0.8),
            s2.animate.shift(RIGHT * explode_dist).set_z(0).set_opacity(0.8),
            s3.animate.shift(UP * explode_dist).set_z(0).set_opacity(0.8),
            s4.animate.shift(LEFT * explode_dist).set_z(0).set_opacity(0.8),
            run_time=1.5, rate_func=rate_functions.ease_in_out_cubic 
        )

        # --- One Triangle ---
        self.play(t_tri_one.animate.set_opacity(1), f_tri_one.animate.set_opacity(1))
        self.play(s1.animate.set_color(YELLOW).set_opacity(1)) 
        self.wait(0.5)
        self.play(s1.animate.set_color(BLUE_D).set_opacity(0.8))

        # --- Count & Calculate (The 4x logic) ---
        self.play(t_tri_four.animate.set_opacity(1))
        
        # Count 1, 2, 3, 4
        for face in [s1, s2, s3, s4]:
            self.play(face.animate.set_color(YELLOW), run_time=0.2)
            self.play(face.animate.set_color(BLUE_D), run_time=0.2)
        
        # Show "4 x 1/2 L l"
        self.add_fixed_in_frame_mobjects(calc_step1)
        self.play(Write(calc_step1))
        self.wait(0.5)
        
        # Highlight "4 x 1/2"
        rect = SurroundingRectangle(calc_step1[0][0:3], color=YELLOW, buff=0.05)
        self.add_fixed_in_frame_mobjects(rect)
        self.play(Create(rect))
        
        # Transform to "2 L l"
        self.add_fixed_in_frame_mobjects(calc_step2)
        self.play(TransformFromCopy(calc_step1, calc_step2), FadeOut(rect))
        self.wait(1)

        # --- Total SA ---
        self.play(t_total.animate.set_opacity(1), f_total.animate.set_opacity(1))
        self.play(Indicate(f_total))
        self.wait(2)
        
        # ================= TRANSITION TO VOLUME =================
        # Clear SA HUD and Exploded Pyramid
        self.play(
            FadeOut(hud_sa), FadeOut(calc_step1), FadeOut(calc_step2),
            FadeOut(s1), FadeOut(s2), FadeOut(s3), FadeOut(s4), FadeOut(base_poly),
            title.animate.become(Text("Volume of Pyramid (आयतन)", font_size=32, color=YELLOW, font="sans-serif").to_corner(UL, buff=0.5))
        )

        # ================= PART 2: VOLUME =================
        # Re-create Pyramid (Ghost view for Volume)
        # Using a new group to control opacity easily
        vol_pyramid = VGroup()
        # Transparent faces
        pts_list = [[v1,v2,v3,v4], [apex,v1,v2], [apex,v2,v3], [apex,v3,v4], [apex,v4,v1]]
        for pts in pts_list:
            vol_pyramid.add(Polygon(*pts, color=GREY, fill_opacity=0.1, stroke_color=WHITE, stroke_width=1))
        
        self.play(Create(vol_pyramid))
        
        # --- Volume HUD ---
        # 1. Formula
        t_vol = Text("Formula:", font_size=24, color=RED, font="sans-serif").next_to(title, DOWN, aligned_edge=LEFT, buff=section_buff)
        f_vol_full = MathTex(r"V = \frac{1}{3} \times \text{Base Area} \times \text{Height}", color=WHITE).scale(hud_scale).next_to(t_vol, DOWN, aligned_edge=LEFT, buff=line_buff)
        f_vol_short = MathTex(r"V = \frac{1}{3} L^2 h", color=YELLOW).scale(0.9).next_to(f_vol_full, DOWN, aligned_edge=LEFT, buff=line_buff+0.2)
        
        hud_vol = VGroup(t_vol, f_vol_full, f_vol_short)
        self.add_fixed_in_frame_mobjects(hud_vol)
        self.play(Write(hud_vol))
        
        # --- 3D Visuals for Volume ---
        # 1. Highlight Base Area (L^2)
        base_highlight = Polygon(v1, v2, v3, v4, color=RED, fill_opacity=0.5, stroke_width=0)
        self.play(FadeIn(base_highlight))
        
        # Label L
        brace_L = Line(v1, v2, color=RED).shift(DOWN*0.2 + OUT*0.2)
        lbl_L = MathTex("L", color=RED).next_to(brace_L, DOWN).rotate(90*DEGREES, axis=RIGHT).add_background_rectangle()
        self.play(Create(brace_L), Write(lbl_L))
        self.wait(0.5)
        
        # 2. Highlight Height (h)
        # Vertical dashed line from apex to center
        h_line = DashedLine(apex, center_base, color=YELLOW, stroke_width=4)
        h_label = MathTex("h", color=YELLOW, font_size=36).next_to(h_line, RIGHT, buff=0.1)
        # Rotate label to face camera
        h_label.rotate(90*DEGREES, axis=RIGHT).rotate(90*DEGREES, axis=OUT)
        h_label.add_background_rectangle()
        
        self.play(Create(h_line), Write(h_label))
        
        # Highlight 'h' in formula
        self.play(Indicate(f_vol_short[0][-1], color=YELLOW, scale_factor=1.5)) # 'h' is usually at end
        
        # Final Camera Move to show depth
        self.move_camera(phi=75 * DEGREES, theta=-10 * DEGREES, run_time=3)
        self.wait(2)

%manim -qk -v warning PyramidAreaAndVolume